Confidence <br/>
![](http://www.sciweavers.org/upload/Tex2Img_1651907171/render.png)

<br/>
Assume:
Data is normal

# Imports & loading

In [1326]:
from statistics import *
from itertools import count

import numpy as np
import scipy
import pandas as pd
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go
pio.templates.default = "plotly_dark"

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 100)
pd.set_option("plotting.backend", "plotly")

def normalize(v):
    norm=np.linalg.norm(v)
    if norm==0:
        norm=np.finfo(v.dtype).eps
    return v/norm

In [1327]:
df_original = pd.read_csv("exp_survey.csv", encoding="utf8")
page = []

# Cleaning

In [1328]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 77 columns):
 #   Column                                                                                              Non-Null Count  Dtype  
---  ------                                                                                              --------------  -----  
 0   Timestamp                                                                                           32 non-null     object 
 1   الجنس                                                                                               32 non-null     object 
 2   التخصص                                                                                              32 non-null     object 
 3   الدفعة                                                                                              32 non-null     object 
 4   أعلى درجة قدرات لك (مثلا: "83")                                                                     32 non-null     in

In [1329]:
df = pd.read_csv("exp_survey.csv", encoding="utf8")

# Filter post-جامعة questions
df = df[df["الجامعة"].isna()]
df.drop("الجامعة", axis=1, inplace=True)

# Manual editing, yikes!
df.at[9, "قدر عدد الجهات اللتي قدمت عليها"] = "15"
df.at[20, "قدر عدد الجهات اللتي قدمت عليها"] = "15"
df.at[20, "قدر عدد الجهات اللتي ردت عليك"] = "6"
df.at[20, "قدر عدد الجهات اللتي ردت عليك بالرفض"] = "4"
df.at[25, "مساحة تعليقات حرة.3"] = np.nan
df.at[26, "ماهي المجالات المهتم فيها"] = np.nan
df.at[22, "ماهي المجالات المهتم فيها"] = np.nan
df.at[28, "ماهي المجالات المهتم فيها"] = "Frontend, Backend, Mobile, Data analysis, Blockchain, ML"
df.at[30, "قدر عدد الجهات اللتي قدمت عليها"] = 10

# Macro filtering
df = (df
    .drop("Timestamp", axis=1)
    .dropna(how="all", axis=1) # Drops empty columns
)

# Columns breakdown 
comments_cols = [col for col in df.columns if "مساحة" in col or "نصائح" in col]
categorical_cols = [col for col in df.columns if "هل" in col or ("كيف" in col and col != "كيف طبيعة التطوع")]
categorical_cols.extend(["الجنس", "التخصص", "الدفعة", "بعد أي مستوى كان التدريب", "كم كان الأجر بالشهر (بالآف)"])
checkbox_cols = [
    "ماهي المجالات المهتم فيها",
    "كيف طبيعة التطوع",
    "في أي المجالات التالية كانت مشاريعك",
    "ما نوع الاسئلة في المقابلة",
    "في أي المجالات التالية كانت تدريبك"
]
number_cols = set(df.columns).difference(set((*checkbox_cols, *categorical_cols, *comments_cols)))


for col in categorical_cols:
    df[col] = df[col].astype("category")


# Expand checkbox cols to multiple boolean cols
for col in checkbox_cols:
    categories = set(val.strip() for val in (",".join(df[col].dropna().values)).split(","))
    for category in categories:
        new_col = f"{col} [{category}]"
        df[new_col] = df[col].apply(lambda x: np.NaN if not isinstance(x, str) else category in x)
    df = df.drop(col, axis=1)


for col in number_cols:
    df[col] = df[col].astype("float32")


renamed_cols = [
    "الجنس",
    "التخصص",
    "الدفعة",
    "القدرات",
    "التحصيلي",
    "المعدل",
    "عدد الصيفيات",
    "عدد الحذف",
    "عدد الحمل",
    "هل غيرت تخصصك",
    "هل تخصصك رغبتك الأولى",
    "هل كان عندك معرفه جيدة بتخصصك",
    "Interest Frontend",
    "Interest Backend",
    "Interest Mobile",
    "Interest Data analysis",
    "Interest Databases",
    "Interest Cyber Security",
    "Interest Academia",
    "Experience Frontend",
    "Experience Backend",
    "Experience Mobile",
    "Experience Data analysis",
    "Experience Databases",
    "Experience Cyber Security",
    "Experience Academia",
    'قدّر ميولك المهني ضد ميولك الأكاديمي',
    'تطوعت سابقا',
    "كيف طبيعة التطوع"
    'قدّر فائدة التطوع على مسيرتك المهنية', 
    'تنصح بالتطوع',
    'مساحة تعليقات حرة.1',
    'عمل سابقا',
    'كيف طبيعة العمل', 
    'كم كان الأجر بالشهر (بالآف)',
    'قدّر فائدة العمل على مسيرتك المهنية', 
    'هل تنصح غيرك بعمل مماثل',
    'هل سبق واشتغلت على مشاريع خاصة فيك في مجال الحاسب',
    'قدّر فائدة مشاريعك ككل بالنسبة للوقت المصروف عليها حسب مسيرتك المهنية',
    'هل كان لمشاريعك الخارجية أثر مباشر على فرصك المهنية (تدريب، توظيف، الخ)',
    'هل تنصح غيرك بالمشاريع الخارجية',
    'هل بحثت سابقا أو تبحث حاليا عن تدريب (التدريب التعاوني)',
    'هل لازلت تبحث عن تدريب', 
    'قدر عدد الجهات اللتي قدمت عليها',
    'قدر عدد الجهات اللتي ردت عليك',
    'قدر عدد الجهات اللتي ردت عليك بالرفض',
    'قدر عدد الجهات اللتي ردت عليك بالقبول',
    'هل قبلت في أي جهة (قبول أولي على الأقل)',
    'هل لديك سيرة ذاتية (CV)',
    'قدر فائدة السيرة الذاتية',
    'نصائح كتابة سيرة ذاتية',
    'هل لديك حساب في Linkedin',
    'قدر فائدة حساب لنكدان',
    'هل لديك حساب في Github',
    'قدر فائدة حساب قتهب',
    'مساحة تعليقات حرة.4', 
    'هل أجريت مقابلة تدريب',
    'ما مدى صعوبة المقابلة بالنسبة لمهاراتك',
    'هل المقابلة تطلبت منك البرمجة', 
    'نصائح قبل المقابلة ',
    'تدرب سابقا', 
    'هل التدريب كان في فصل صيفي',
    'بعد أي مستوى كان التدريب',
    'ما مدى صعوبة التدريب بالنسبة لمهاراتك',
    'مدة التدريب',
    'تدريب بسبب علاقات',
    'وظيفة بعد التدريب', 
    'مساحة تعليقات حرة.5',
    'Interests Game dev',
    'Interests Cyber Security',
    'Interests Mobile',
    'Interests UX Design',
    'Interests Software testing',
    'Interests Product management',
    'Interests Frontend',
    'Interests Graphics',
    'Interests Databases',
    'Interests Data analysis',
    'Interests Data engineering',
    'Interests Blockchain',
    'Interests ML',
    'Interests business analysis',
    'Interests Backend',
    'Interests Academia',
    'تطوع نادي', 
    'تطوع خارجي',
    'Projects Game dev',
    'Projects Cyber Security',
    'Projects Algorithm design',
    'Projects UX Design',
    'Projects game development',
    'Projects Frontend',
    'Projects Mobile app',
    'Projects Blockchain + ML',
    'Projects Databases',
    'Projects Data analysis',
    'Projects Backend',
    'Projects Academia',
    'اسئلة تخصصات دقيقة',
    'اسئلة حاسب عامة',
    'اسئلة موارد بشرية',
    'Internship Cyber Security',
    'Internship Quality Assurance',
    'Internship Product management',
    'Internship Databases',
    'Internship Data analysis',
    'Internship ML',
]
name_map = {old:new for old, new in zip(df.columns[:len(renamed_cols)], renamed_cols)}
comments_cols = [name_map.get(col, col) for col in comments_cols]
categorical_cols = [name_map.get(col, col) for col in categorical_cols]
checkbox_cols = [name_map.get(col, col) for col in checkbox_cols]
number_cols = [name_map.get(col, col) for col in number_cols]
df.columns = [name_map.get(col, col) for col in df.columns]

# Sort columns
df = df.reindex(sorted(df.columns), axis=1)

#### Define new cols

In [1330]:
df["التصنيف"] = pd.cut(df["المعدل"], [0, 2, 2.75, 3.75, 4.5, 5], labels=["راسب", "مقبول", "جيد", "جيد جدا", "ممتاز"])

df["معدل القبول"] = df["قدر عدد الجهات اللتي ردت عليك بالقبول"] / df["قدر عدد الجهات اللتي قدمت عليها"]
df["معدل الرد"] = df["قدر عدد الجهات اللتي ردت عليك"] / df["قدر عدد الجهات اللتي قدمت عليها"]

# Meta analysis

#### Demographic

In [1331]:
fig = px.sunburst(
    df, 
    path=["الجنس", "الدفعة", "التخصص", "التصنيف"], 
    values="المعدل",
    color="الدفعة",
    title="ديموغرافية الإستبانة",
    width=1100, 
    height=1100
)
fig.update_layout(
    font=dict(
        size=15,
    )
)
fig.update_traces(textinfo="label+percent parent",)
page.append(fig)
fig


#### Survey structure
###### What are the different sections on a survey

#### Counts analysis
###### How many people particiapted in each section

In [1332]:
section_questions = [
    "تطوعت سابقا",
    "عمل سابقا",
    "هل سبق واشتغلت على مشاريع خاصة فيك في مجال الحاسب",
    "هل بحثت سابقا أو تبحث حاليا عن تدريب (التدريب التعاوني)",
    "هل لازلت تبحث عن تدريب",
    "هل قبلت في أي جهة (قبول أولي على الأقل)",
    "هل أجريت مقابلة تدريب",
    "تدرب سابقا",
]

In [1333]:
temp = (df[section_questions] == "نعم").sum()
question2num_answers = dict(zip(temp.index, temp.values))
question2num_answers

{'تطوعت سابقا': 14,
 'عمل سابقا': 7,
 'هل سبق واشتغلت على مشاريع خاصة فيك في مجال الحاسب': 17,
 'هل بحثت سابقا أو تبحث حاليا عن تدريب (التدريب التعاوني)': 19,
 'هل لازلت تبحث عن تدريب': 13,
 'هل قبلت في أي جهة (قبول أولي على الأقل)': 11,
 'هل أجريت مقابلة تدريب': 10,
 'تدرب سابقا': 5}

In [1366]:
sections = dict(
    sections=[
        "كل الأقسام", 
        "قسم النشاطات اللاصفية", 
        "قسم التطوع", 
        "قسم العمل", 
        "قسم المشاريع", 
        "قسم التدريب", 
        "قسم  المقبولين (قبول أولي على الأقل) في التدريب", 
        "قسم من أجروا المقابلة",
        "قسم المتدربين سابقا",
        ],
    parents=[
        "",
        "كل الأقسام",
        "قسم النشاطات اللاصفية",
        "قسم النشاطات اللاصفية",
        "قسم النشاطات اللاصفية",
        "كل الأقسام",
        "قسم التدريب",
        "قسم  المقبولين (قبول أولي على الأقل) في التدريب",
        "قسم التدريب",
        ],
    participants=[
        len(df),
        sum(df[["تطوعت سابقا", "عمل سابقا", "هل سبق واشتغلت على مشاريع خاصة فيك في مجال الحاسب"]].apply(lambda row: 1 if "نعم" in row.values else 0, axis=1)),
        question2num_answers["تطوعت سابقا"],
        question2num_answers["عمل سابقا"],
        question2num_answers["هل سبق واشتغلت على مشاريع خاصة فيك في مجال الحاسب"],
        question2num_answers["هل بحثت سابقا أو تبحث حاليا عن تدريب (التدريب التعاوني)"],
        question2num_answers["هل قبلت في أي جهة (قبول أولي على الأقل)"],
        question2num_answers["هل أجريت مقابلة تدريب"],
        question2num_answers["تدرب سابقا"],
    ]
)

fig =px.icicle(
    sections,
    names="sections",
    parents="parents",
    values="participants",
    color='participants'
)
fig.update_traces(
            tiling = dict(
            orientation='v',
            flip='x'
        )
)
fig.update_layout(
    height=500,
    width=1400,
    font=dict(
        family="Arial",
        size=20,
    )
)
fig.update_traces(textinfo="label+percent root")
page.append(fig)
fig

# Analysis

#### Utilities

In [1335]:
# Utility for interests and experiences
interests_cols = [col for col in df.columns if "Interest " in col]
experiences_cols = [col for col in df.columns if "Experience" in col]
field_names = interests_cols # [col[col.find("[")+1 : col.rfind("]")] for col in interests_cols]
applying_cols = ["قدر عدد الجهات اللتي قدمت عليها", "قدر عدد الجهات اللتي ردت عليك", "قدر عدد الجهات اللتي ردت عليك بالرفض", "قدر عدد الجهات اللتي ردت عليك بالقبول", "معدل القبول", "معدل الرد"]
grade_cols = ["المعدل", "التحصيلي", "القدرات"]
social_media_cols = [
    "قدر فائدة السيرة الذاتية",
    "قدر فائدة حساب لنكدان",
    "قدر فائدة حساب قتهب",
]

df_ex = df[experiences_cols]
df_int = df[interests_cols]
df_ex.columns = field_names
df_int.columns = field_names
df_int.loc[:, "Type"] = "Interest"
df_ex.loc[:, "Type"] = "Experience"
df_int_ex = pd.concat([df_int, df_ex])

df_ex_sum = df[experiences_cols].apply(sum)
df_int_sum = df[interests_cols].apply(sum)

# Df of grades, type of grade and applying cols
grades_df = df.melt(value_vars=["التحصيلي", "القدرات"], var_name="Type", value_name="Grade")
grades_df[applying_cols] = pd.concat([df[applying_cols], df[applying_cols]]).reset_index(drop=True)

d:\Softwarez\Anaconda\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

d:\Softwarez\Anaconda\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [1336]:
#  Interests and experiences in one table
df_int_ex_concat = pd.concat([df_int_sum, df_ex_sum])
data = zip(*[field_names * 2, df_int_ex_concat.values / max(df_int_ex_concat.values), ["Interest"] * len(field_names) + ["Experience"] * len(field_names)])
df_int_ex_sum = pd.DataFrame(data, columns=["Field", "Score", "Type"])


#### Grades analysis
###### Distribution of all grades 

In [1337]:
fig = px.histogram(
    df[["القدرات", "التحصيلي"]], 
    marginal="rug",
    nbins=20,
    text_auto=True,
    width=1000,
    height=700,

    title="توزيع درجات القياس",
)
page.append(fig)
fig

In [1338]:
fig = px.histogram(
    df["المعدل"], 
    marginal="rug", 
    nbins=20,
    text_auto=True,
    width=1000, 
    height=700, 
    title="توزيع المعدلات"
)
fig.update_layout(
    xaxis_title="المعدل",
    yaxis_title="العدد",
)
page.append(fig)
fig

In [1339]:
temp = df[["القدرات", "التحصيلي"]].copy()
temp.index = df["المعدل"]
fig = px.scatter(
    temp, 
    trendline="ols", 
    width=1000,
    height=700,
    title="إرتباط المعدل مع القدرات والتحصيلي"
)
fig.update_layout(
    xaxis_title="المعدل",
    yaxis_title="درجة القياس",
)
page.append(fig)
fig

#### Interest vs. experience

31.0

In [1359]:
fig = px.violin(
    df_int_ex,
    color="Type",
    violinmode="group",
    title=f"إختلاف الإهتمام والخبرة لكل تخصص دقيق. الردود={int(len(df_int_ex) / 2)}",
    width=1500,
    height=500,
)
fig.update_layout(
    xaxis_title="",
    yaxis_title="التقييم من 0 حتى 5",
)
page.append(fig)
fig

In [1341]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_titles=["علاقة الدرجات مع معدل القبول", "علاقة الدرجات مع معدل الرد"])
fig.add_traces(
    px.scatter(
        grades_df,
        x='Grade', 
        y="معدل القبول",
        color="Type",
        trendline="ols", 
    ).data,
    rows=1, 
    cols=1
).add_traces(
    px.scatter(
        grades_df,
        x='Grade', 
        y="معدل الرد",
        color="Type",
        trendline="ols", 
    ).data,
    rows=2, 
    cols=1,
)
fig.update_layout(
    height=1000, 
    width=900
)
page.append(fig)
fig

In [1342]:
fig = px.scatter(
    df,
    x='المعدل', 
    y="معدل القبول", 
    trendline="ols", 
    width=700,
    height=700,
)
fig

#### Correlation between variables

In [1343]:
fig = px.imshow(df.corr())
fig.update_layout(
    width=1500, 
    height=1000
)
page.append(fig)
fig

### Jobs and volenteering

In [1344]:
temp_table = pd.crosstab(df["عمل سابقا"], df["تطوعت سابقا"])
page.append(temp_table)
temp_table

تطوعت سابقا,لا,نعم
عمل سابقا,,
لا,15,9
نعم,2,5


In [1356]:
money_counts = df.groupby("كم كان الأجر بالشهر (بالآف)")["كم كان الأجر بالشهر (بالآف)"].count()
fig = px.pie(
    names=money_counts.index, 
    values=money_counts,
    width=400,
    height=400,
    hole=.92,
    title=f"راتب العمل اثناء الدراسة. الردود={nepotism_counts.values.sum()}",
)
page.append(fig)
fig

### Internships

In [1346]:
fig = px.box(
    df.melt(value_vars=applying_cols[:4], var_name="المتغير", value_name="العدد"), 
    y="العدد",
    color="المتغير",
    title="متغيرات التدريب",
    width=700,
    height=1000
)
page.append(fig)
fig

In [1347]:
social_media_cols[1]

'قدر فائدة حساب لنكدان'

In [1348]:
social_media_df = pd.concat([
    df.groupby(["قدر فائدة السيرة الذاتية"]).size(),
    df.groupby(["قدر فائدة حساب لنكدان"]).size(),
    df.groupby(["قدر فائدة حساب قتهب"]).size(), 
    ], axis=1).fillna(0)
social_media_df.columns = social_media_cols
social_media_df.index = social_media_df.reset_index()["index"].apply(lambda n: "⭐ " * int(n) if n else "بلا أهمية")

fig = px.bar(
    social_media_df, 
    barmode="stack",
    orientation="h",
    width=600, 
    height=800, 
    title="تقدير أهمية حسابات التواصل الإجتماعي", 
)
fig.update_layout(
    yaxis_automargin=False,
    yaxis_title_standoff=500,
    yaxis_title="التفدير",
    xaxis_title="عدد التقاييم",
    margin={"l": 150}
)
page.append(fig)
fig

In [1349]:
fig = px.bar(
    df.corr()[grade_cols].T[applying_cols], 
    barmode="group",
    orientation="h",
    title="ارتباط الأداء الدراسي مع احصائيات التدريب الميداني")
fig.update_layout(
    width=800, 
    height=1500, 
    xaxis_title="الأرتباط",
    yaxis_title="",
)
page.append(fig)
fig

In [1350]:
nepotism_counts = df.groupby("تدريب بسبب علاقات")["تدريب بسبب علاقات"].count()
fig = px.pie(
    names=nepotism_counts.index,
    values=nepotism_counts,
    width=400,
    height=400,
    hole=.92,
    title=f"تدربت بسبب علاقات. الردود={nepotism_counts.values.sum()}"

)
page.append(fig)
fig

In [1351]:
temp = df.groupby("وظيفة بعد التدريب")["وظيفة بعد التدريب"].count()
fig = px.pie(
    names=temp.index,
    values=temp,
    width=400,
    height=400,
    hole=.92,
    title=f"عرض عليه وظيفة بعد التدريب. الردود={temp.values.sum()}"
)
page.append(fig)
fig

In [1352]:
temp_table = pd.pivot_table(
    df, 
    values=applying_cols, 
    index=["عمل سابقا", "تطوعت سابقا",],
    aggfunc="mean"
).style.background_gradient(cmap='autumn_r')
page.append(temp_table.data)
temp_table

# Export to HTML

In [1353]:
with open("index_empty.html.test") as file:
    html = file.read()

In [1354]:
with open("experience.html", "w", encoding="utf-8") as f:
    f.write(html)
    for content in page:
        if isinstance(content, pd.DataFrame):
            f.write(content.to_html())
        else:
            pass
            f.write(content.to_html(full_html=False, include_plotlyjs="cdn"))

In [1355]:
list(img.write_image(f"images/{idx}.png") for idx, img in zip(count(0), page) if isinstance(img, type(page[0])))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]